# Import and Read data


In [4]:
import sys
sys.path.append('.')  # Add current directory to path
from build_graph_and_train import *

In [5]:
partition = 200

In [6]:
df = pd.read_csv(f"../../../data/top30groups/LongLatCombined/combined/combined{partition}.csv")

In [7]:
from sklearn.preprocessing import StandardScaler

# Columns to exclude from scaling
exclude_cols = ['gname', 'longitude', 'latitude']

# Columns to scale
scale_cols = [col for col in df.columns if col not in exclude_cols]

# Scale only selected columns
scaler = StandardScaler()
df[scale_cols] = scaler.fit_transform(df[scale_cols])

In [8]:
import os 
if not os.path.isdir(f"Results{partition}"):
    os.mkdir(f"Results{partition}")

# Create longlat feature

In [9]:
geodata = ['longitude', 'latitude']
combined_geo = df.copy()
combined_geo['longlat'] = list(zip(df['longitude'], df['latitude']))
combined_geo = combined_geo.drop(columns=geodata)

In [10]:
import ast

def to_tuple_if_needed(val):
    if isinstance(val, str):
        return ast.literal_eval(val)
    return val  # already a tuple

combined_geo['longlat'] = combined_geo['longlat'].apply(to_tuple_if_needed)

# Weapon type prediction

In [11]:
label_index = {g: i for i, g in enumerate(sorted(df['gname'].unique()))}
continuous_cols = ['weaptype1', 'nkill', 'targtype1', 'attacktype1']
y_preds = []
y_trues = []
for i in range(len(continuous_cols)):
    data, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, row_to_node_index, index_to_label = build_graph_data(combined_geo, label_index, continuous_col=continuous_cols[i])
    args = {
        'partition': f"gtd{partition}",
        'embed_dim': 16,
        'lr': 0.01,
        'epochs': 200,
        'feat_dropout': 0,
        'n_tree': 80,
        'tree_depth': 10,
        'tree_feature_rate': 0.5,
        'n_class': len(label_index)
    }
    best_acc, best_epoch, best_precision, best_recall, best_f1, y_pred_decoded, y_true_decoded = train_joint(data, data.edge_index, y_gcn, y_nrf, non_geo_features, train_mask, test_mask, args, row_to_node_index, index_to_label)
    y_preds.append(y_pred_decoded)
    y_trues.append(y_true_decoded)
    
    with open(f"Results{partition}/Results_{continuous_cols[i]}_prediction", "w") as f:
        f.write(f"Best acc: {best_acc} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best recall: {best_recall} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best precision: {best_precision} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")
        f.write(f"Best f1: {best_f1} in epoch {best_epoch} for {continuous_cols[i]} prediction\n")

hej


/home/jovyan/MEX0512/GTD_2025/Codes/Baselines/GraphRfi_OneHotLongLat/build_graph_and_train.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 395.4909 | NRF Loss: 3.4006 | JOINT Loss: 398.8915 | NRF Acc: 0.0380
Epoch 02 | GCN MSE Loss: 278.3042 | NRF Loss: 3.3971 | JOINT Loss: 281.7013 | NRF Acc: 0.0766
Epoch 03 | GCN MSE Loss: 184.2389 | NRF Loss: 3.3965 | JOINT Loss: 187.6354 | NRF Acc: 0.0900
Epoch 04 | GCN MSE Loss: 112.3945 | NRF Loss: 3.3920 | JOINT Loss: 115.7865 | NRF Acc: 0.0793
Epoch 05 | GCN MSE Loss: 61.6139 | NRF Loss: 3.3845 | JOINT Loss: 64.9984 | NRF Acc: 0.0803
Epoch 06 | GCN MSE Loss: 29.9538 | NRF Loss: 3.3757 | JOINT Loss: 33.3295 | NRF Acc: 0.0793
Epoch 07 | GCN MSE Loss: 14.3802 | NRF Loss: 3.3800 | JOINT Loss: 17.7602 | NRF Acc: 0.0659
Epoch 08 | GCN MSE Loss: 10.9246 | NRF Loss: 3.3916 | JOINT Loss: 14.3162 | NRF Acc: 0.0605
Epoch 09 | GCN MSE Loss: 15.0950 | NRF Loss: 3.4030 | JOINT Loss: 18.4980 | NRF Acc: 0.0595
Epoch 10 | GCN MSE Loss: 22.8051 | NRF Loss: 3.4002 | JOINT Loss: 26.2054 | NRF Acc: 0.0659
Epoch 11 | GCN MSE Loss: 30.9788 | NRF Loss: 3.3862 | JOINT Loss: 34.365

/home/jovyan/MEX0512/GTD_2025/Codes/Baselines/GraphRfi_OneHotLongLat/build_graph_and_train.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 1586.0627 | NRF Loss: 3.4044 | JOINT Loss: 1589.4670 | NRF Acc: 0.0412
Epoch 02 | GCN MSE Loss: 1314.5513 | NRF Loss: 3.4070 | JOINT Loss: 1317.9584 | NRF Acc: 0.0458
Epoch 03 | GCN MSE Loss: 1075.7402 | NRF Loss: 3.4061 | JOINT Loss: 1079.1464 | NRF Acc: 0.0806
Epoch 04 | GCN MSE Loss: 868.1438 | NRF Loss: 3.4075 | JOINT Loss: 871.5513 | NRF Acc: 0.0766
Epoch 05 | GCN MSE Loss: 690.1127 | NRF Loss: 3.4052 | JOINT Loss: 693.5179 | NRF Acc: 0.0428
Epoch 06 | GCN MSE Loss: 539.5573 | NRF Loss: 3.4068 | JOINT Loss: 542.9641 | NRF Acc: 0.0404
Epoch 07 | GCN MSE Loss: 413.8240 | NRF Loss: 3.4091 | JOINT Loss: 417.2331 | NRF Acc: 0.0469
Epoch 08 | GCN MSE Loss: 310.5450 | NRF Loss: 3.4068 | JOINT Loss: 313.9519 | NRF Acc: 0.0683
Epoch 09 | GCN MSE Loss: 227.1351 | NRF Loss: 3.3987 | JOINT Loss: 230.5339 | NRF Acc: 0.0777
Epoch 10 | GCN MSE Loss: 161.2057 | NRF Loss: 3.3950 | JOINT Loss: 164.6007 | NRF Acc: 0.0753
Epoch 11 | GCN MSE Loss: 110.5600 | NRF Loss: 3.3923 |

/home/jovyan/MEX0512/GTD_2025/Codes/Baselines/GraphRfi_OneHotLongLat/build_graph_and_train.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 314.7065 | NRF Loss: 3.4014 | JOINT Loss: 318.1079 | NRF Acc: 0.0538
Epoch 02 | GCN MSE Loss: 223.8621 | NRF Loss: 3.3931 | JOINT Loss: 227.2551 | NRF Acc: 0.1069
Epoch 03 | GCN MSE Loss: 155.8006 | NRF Loss: 3.4064 | JOINT Loss: 159.2070 | NRF Acc: 0.0876
Epoch 04 | GCN MSE Loss: 108.3169 | NRF Loss: 3.4050 | JOINT Loss: 111.7219 | NRF Acc: 0.1095
Epoch 05 | GCN MSE Loss: 78.1123 | NRF Loss: 3.4018 | JOINT Loss: 81.5141 | NRF Acc: 0.1754
Epoch 06 | GCN MSE Loss: 59.5368 | NRF Loss: 3.3858 | JOINT Loss: 62.9225 | NRF Acc: 0.1082
Epoch 07 | GCN MSE Loss: 46.1077 | NRF Loss: 3.3765 | JOINT Loss: 49.4842 | NRF Acc: 0.1168
Epoch 08 | GCN MSE Loss: 34.5344 | NRF Loss: 3.3645 | JOINT Loss: 37.8989 | NRF Acc: 0.1545
Epoch 09 | GCN MSE Loss: 24.5774 | NRF Loss: 3.3497 | JOINT Loss: 27.9271 | NRF Acc: 0.1698
Epoch 10 | GCN MSE Loss: 17.3301 | NRF Loss: 3.3356 | JOINT Loss: 20.6657 | NRF Acc: 0.1427
Epoch 11 | GCN MSE Loss: 13.6674 | NRF Loss: 3.3221 | JOINT Loss: 16.989

/home/jovyan/MEX0512/GTD_2025/Codes/Baselines/GraphRfi_OneHotLongLat/build_graph_and_train.py:123: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_nrf = torch.tensor(y_nrf, dtype=torch.long).to(device)


Epoch 01 | GCN MSE Loss: 106.5873 | NRF Loss: 3.4009 | JOINT Loss: 109.9881 | NRF Acc: 0.0608
Epoch 02 | GCN MSE Loss: 67.2534 | NRF Loss: 3.3773 | JOINT Loss: 70.6307 | NRF Acc: 0.0886
Epoch 03 | GCN MSE Loss: 45.6142 | NRF Loss: 3.3798 | JOINT Loss: 48.9940 | NRF Acc: 0.0520
Epoch 04 | GCN MSE Loss: 36.3451 | NRF Loss: 3.3812 | JOINT Loss: 39.7264 | NRF Acc: 0.1323
Epoch 05 | GCN MSE Loss: 31.5646 | NRF Loss: 3.3839 | JOINT Loss: 34.9485 | NRF Acc: 0.1219
Epoch 06 | GCN MSE Loss: 27.2287 | NRF Loss: 3.3781 | JOINT Loss: 30.6068 | NRF Acc: 0.1127
Epoch 07 | GCN MSE Loss: 22.8706 | NRF Loss: 3.3715 | JOINT Loss: 26.2421 | NRF Acc: 0.1034
Epoch 08 | GCN MSE Loss: 19.3340 | NRF Loss: 3.3617 | JOINT Loss: 22.6957 | NRF Acc: 0.1385
Epoch 09 | GCN MSE Loss: 16.9646 | NRF Loss: 3.3500 | JOINT Loss: 20.3146 | NRF Acc: 0.1583
Epoch 10 | GCN MSE Loss: 15.5600 | NRF Loss: 3.3362 | JOINT Loss: 18.8962 | NRF Acc: 0.1513
Epoch 11 | GCN MSE Loss: 14.5172 | NRF Loss: 3.3246 | JOINT Loss: 17.8417 | NR

In [12]:
def plot_confusion_matrix(y_true, y_pred, labels, continuous_col):
    from sklearn.metrics import confusion_matrix
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np

    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    # Save the figure
    save_path = f"Results{partition}/cm_{partition}_{continuous_col}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition {partition} to {save_path}")


In [13]:
for i in range(len(continuous_cols)):
    plot_confusion_matrix(y_preds[i], y_trues[i], sorted(df['gname'].unique()), continuous_cols[i])

Saved confusion matrix for partition 200 to Results200/cm_200_weaptype1.png
Saved confusion matrix for partition 200 to Results200/cm_200_nkill.png
Saved confusion matrix for partition 200 to Results200/cm_200_targtype1.png
Saved confusion matrix for partition 200 to Results200/cm_200_attacktype1.png
